###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [ ]:
import pandas as pd

###Archivo 9vha-vh9n.csv

Contiene información del delito de hurto en Colombia a través de las modalidades de motocicletas y automotores desde 01 de enero del año 2010 al 30 de abril del año 2024.

In [ ]:
df = pd.read_csv("https://www.datos.gov.co/resource/9vha-vh9n.csv?$limit=500000", dtype={'codigo_dane': str})
df.head()

,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,genero,grupo_etario,tipo_de_hurto,cantidad
0,ANTIOQUIA,MEDELLÍN (CT),05001000,ARMA DE FUEGO,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
1,ANTIOQUIA,COPACABANA,05212000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
2,ANTIOQUIA,MEDELLÍN (CT),05001000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
3,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
4,VALLE,CALI (CT),76001000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376433 entries, 0 to 376432
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   departamento   376433 non-null  object
 1   municipio      376433 non-null  object
 2   codigo_dane    376433 non-null  object
 3   armas_medios   376433 non-null  object
 4   fecha_hecho    376433 non-null  object
 5   genero         376433 non-null  object
 6   grupo_etario   376433 non-null  object
 7   tipo_de_hurto  376433 non-null  object
 8   cantidad       376433 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 25.8+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio', 'fecha_hecho', 'genero', 'grupo_etario', 'tipo_de_hurto','cantidad']
vehicle_theft = df[relevant_cols]
vehicle_theft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376433 entries, 0 to 376432
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   codigo_dane    376433 non-null  object
 1   departamento   376433 non-null  object
 2   municipio      376433 non-null  object
 3   fecha_hecho    376433 non-null  object
 4   genero         376433 non-null  object
 5   grupo_etario   376433 non-null  object
 6   tipo_de_hurto  376433 non-null  object
 7   cantidad       376433 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 23.0+ MB


### Convertir columna fecha_hecho a tipo date

In [ ]:
# Hacer una copia explícita del DataFrame
vehicle_theft_ = vehicle_theft.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
vehicle_theft_['fecha_hecho'] = pd.to_datetime(vehicle_theft_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
vehicle_theft_.loc[:, 'fecha_hecho'] = vehicle_theft_['fecha_hecho'].dt.year

### Verificar valores nulos

In [ ]:
vehicle_theft_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,0
genero,0
grupo_etario,0
tipo_de_hurto,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['genero', 'grupo_etario', 'tipo_de_hurto', 'departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(vehicle_theft_[column].unique())
    print()


Categorías en la columna 'genero':
['NO APLICA' 'NO REPORTADO']

Categorías en la columna 'grupo_etario':
['NO APLICA' 'NO REPORTADO']

Categorías en la columna 'tipo_de_hurto':
['HURTO AUTOMOTORES' 'HURTO MOTOCICLETAS']

Categorías en la columna 'departamento':
['ANTIOQUIA' 'CUNDINAMARCA' 'VALLE' 'CALDAS' 'CESAR' 'NORTE DE SANTANDER'
 'BOYACÁ' 'CAUCA' 'ATLÁNTICO' 'SANTANDER' 'CAQUETÁ' 'QUINDÍO' 'MAGDALENA'
 'META' 'SUCRE' 'GUAJIRA' 'TOLIMA' 'NARIÑO' 'RISARALDA' 'BOLÍVAR' 'HUILA'
 'CASANARE' 'ARAUCA' 'PUTUMAYO' 'CÓRDOBA' 'GUAVIARE' 'VICHADA' 'CHOCÓ'
 'SAN ANDRÉS' 'AMAZONAS' 'VAUPÉS' 'GUAINÍA']

Categorías en la columna 'municipio':
['MEDELLÍN (CT)' 'COPACABANA' 'BOGOTÁ D.C. (CT)' ... 'Aratoca'
 'Belalcázar' 'La Salina']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    vehicle_theft_[col] = vehicle_theft_[col].apply(remove_accents_and_special_chars)

- Mejorar consistencia de las columnas 'genero' y 'grupo_etario'

Estas dos columnas solo contienen categorías irrelevantes, por lo tanto las suprimimos

In [ ]:
# Imprime categorías de columnas
print(f"Categorías columna genero: {vehicle_theft_['genero'].unique()}")
print(f"Categorías columna grupo_etario: {vehicle_theft_['genero'].unique()}")

Categorías columna genero: ['NO APLICA' 'NO REPORTADO']
Categorías columna grupo_etario: ['NO APLICA' 'NO REPORTADO']


In [ ]:
# Eliminar columnas genero y grupo_etario
vehicle_theft_ = vehicle_theft_.drop(['genero', 'grupo_etario'], axis=1)

- Codificación de algunas variables categóricas

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df sexual_crimes_

In [ ]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
vehicle_theft_['codigo_dane'] = vehicle_theft_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = vehicle_theft_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [ ]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [ 8  7 27 13 12 19]


In [ ]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_digitos_codigo_dane,count,percentage
0,8,331396,0.880359
1,7,38546,0.102398
2,13,3138,0.008336
3,19,3058,0.008124
4,12,150,0.000398


In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = vehicle_theft_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
  codigo_dane
0    05001000
1    05212000
2    05001000
3    11001000
4    76001000

Muestra de registros con longitud 7:
      codigo_dane
65471     5088000
65472     5088000
65473     5001000
65486     5154000
65487     5001000

Muestra de registros con longitud 13:
          codigo_dane
358144  ARMA DE FUEGO
358145  ARMA DE FUEGO
358146  ARMA DE FUEGO
358147  ARMA DE FUEGO
358148  ARMA DE FUEGO

Muestra de registros con longitud 19:
                codigo_dane
361385  SIN EMPLEO DE ARMAS
361386  SIN EMPLEO DE ARMAS
361387  SIN EMPLEO DE ARMAS
361388  SIN EMPLEO DE ARMAS
361389  SIN EMPLEO DE ARMAS

Muestra de registros con longitud 12:
         codigo_dane
360095  CONTUNDENTES
360096  CONTUNDENTES
360097  CONTUNDENTES
360098  CONTUNDENTES
360099  CONTUNDENTES

Muestra de registros con longitud 27:
                        codigo_dane
357999  ARMA BLANCA / CORTOPUNZANTE
358000  ARMA BLANCA / CORTOPUNZANTE
358001  ARMA BLANCA / CORTOPUNZANTE
358002 

### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 88% tiene 8 dígitos (al parecer se le adicionaron 3 ceeros al final del código que realmente es de 5 dígitos), el 10% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió), el restante porcentaje que no llega al 2% de los códigos en vez del código aparece una cadena de 12, 13, 19 o 27 caracteres, claramente un error

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [ ]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = vehicle_theft_[vehicle_theft_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [ ]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
vehicle_theft_['codigo_dane'] = vehicle_theft_['codigo_dane'].apply(add_zero_if_length_7)

In [ ]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(vehicle_theft_[vehicle_theft_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [ ]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
vehicle_theft_['codigo_dane'] = vehicle_theft_['codigo_dane'].apply(trim_last_3_if_length_8)

In [ ]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(vehicle_theft_[vehicle_theft_['codigo_dane'].str.len() == 8])

0

### Solucionar los codigo_dane que en el dataset fueron digitados erroneamente con diferentes palabras

  a) Iniciamos ajustando la columna de nombre de departamento de los dataframe: sexual_crimes_ y dept_mpios_codes para que coincidan en formato

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños en el df: dept_mpios_codes

In [ ]:
# Aplicar funcion a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(vehicle_theft_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en sexual_crimes_", "Departamentos dept_mpios_codes")

Departamentos en sexual_crimes_ que no están en Departamentos dept_mpios_codes:
{'GUAJIRA', 'VALLE', 'SAN ANDRES'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe vehicle_theft_
vehicle_theft_['departamento'] = vehicle_theft_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(vehicle_theft_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en sexual_crimes_", "Departamentos dept_mpios_codes")

Departamentos en sexual_crimes_ que no están en Departamentos dept_mpios_codes:
set()


  b) Aplicar coincidencia difusa para combinacion departamento + municipio en los dos dataframe

Realizar coincidencia difusa (fuzzy matching), significa que puede comparar dos cadenas de texto (strings) y medir su similitud, incluso si no son exactamente iguales. Esta técnica es muy útil en este caso porque si bien los nombres de los departamentos estan debidamente ajustados en los dos df,  los nombres de los municipios pueden tener diferencias (errores de tipeo, variantes en nombres, etc.).

El utilizar la columna del nombre del departamento, es importante en este caso porque en colombia existen municipios con el mismo nombre.


 - Instalar la thefuzz en el entorno de google colab, para realizar coincidencia difusa

In [ ]:
!pip install thefuzz

- Aplicar thefuzz para coincidencia difusa

In [ ]:
# Crear una clave única de departamento + municipio en ambos datasets
vehicle_theft_['dept_mpio'] = vehicle_theft_['departamento'] + '_' + vehicle_theft_['municipio']
dept_mpios_codes['dept_mpio'] = dept_mpios_codes['dept_name'] + '_' + dept_mpios_codes['mupio_name']

In [ ]:
# Crear un diccionario de municipios y códigos basado en dept_mpios_codes
municipios_dict = dict(zip(dept_mpios_codes['dept_mpio'], dept_mpios_codes['dept_mpio_code']))

In [ ]:
from thefuzz import process

# Funcion para Usar fuzzy matching (thefuzz)
def get_best_match(row, municipios_dict, threshold=80):
    dept_mpio_sexual = row['departamento'] + '_' + row['municipio']

    # Buscar la mejor coincidencia en dept_mpios_codes usando fuzzy matching
    best_match, score = process.extractOne(dept_mpio_sexual, municipios_dict.keys())

    # Si la similitud supera el umbral definido, devolver el código mapeado
    if score >= threshold:
        return municipios_dict[best_match]
    else:
        return row['codigo_dane']  # Mantener el código original si no hay coincidencia segura


In [ ]:
# Aplicar la función solo a las filas donde el 'codigo_dane' no tenga 5 dígitos
vehicle_theft_['codigo_dane_corr'] = vehicle_theft_.apply(
    lambda row: get_best_match(row, municipios_dict, threshold=80) if len(row['codigo_dane']) != 5 else row['codigo_dane'],
    axis=1
)

  - Verificar resultados obtenidos

In [ ]:
vehicle_theft_['codigo_dane_corr'].apply(len).nunique()

1

In [ ]:
vehicle_theft_['codigo_dane_corr'].apply(len).value_counts()

,count
codigo_dane_corr,
5,376433


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de codigos
compare_lists(vehicle_theft_['codigo_dane_corr'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en sexual_crimes_", "Códigos de municipios dept_mpios_codes")

Departamentos en sexual_crimes_ que no están en Departamentos dept_mpios_codes:
set()


  Nota: El resultado de conjunto vacio indica que todos los codigo_dane en el dataframe vehiclde_theft corresponden a codigos reales definidos en el df dept_mpios_codes

### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['codigo_dane', 'departamento', 'municipio', 'dept_mpio']
final_vehicle_theft = vehicle_theft_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_vehicle_theft['source_id'] = 7

In [ ]:
final_vehicle_theft.columns

Index(['fecha_hecho', 'tipo_de_hurto', 'cantidad', 'codigo_dane_corr',
       'source_id'],
      dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'delito': 'crime_type',
    'codigo_dane_corr': 'dane_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_vehicle_theft.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_vehicle_theft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376433 entries, 0 to 376432
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   year_of_incident  376433 non-null  int32 
 1   tipo_de_hurto     376433 non-null  object
 2   amount            376433 non-null  int64 
 3   dane_code         376433 non-null  object
 4   source_id         376433 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 12.9+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_vehicle_theft.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/vehicle_theft.csv', index=False)